### classify newsgroup docsby considering each document a simple mean of its individual word embeddings, using one-vs-all SVM (Linear, RBF)

In [1]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from IPython.display import SVG

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
SEED=np.random.randint(1,1000)
SEED

795

In [4]:
np.random.seed(seed=SEED)

In [5]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [6]:
preprocessed_texts = texts

In [7]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [9]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 "it's",
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',
 'to

In [10]:
# word => embedding
embeddings_index = files_helper.read_glove_wiki(d=EMBEDDING_DIM)

In [11]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
        
embedding_matrix.shape        

(174075, 100)

In [13]:
num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = True))

# no need for extra layers if we aren't training the pooling layers
model.add(GlobalAvgPool1D(name='avg_pooling'))
model.add(Dense(64,activation='relu',name='dense_1'))
model.add(Dropout(0.5,name="dropout_1"))
model.add(Dense(64,activation='relu',name="dense_2"))
model.add(Dropout(0.5,name="dropout_2"))
model.add(Dense(num_labels, activation='softmax',name="softmax_output"))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [14]:
%%time

model.fit(X_train,Y_train, validation_data=(X_val, Y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 25s - loss: 2.8899 - acc: 0.0884 - val_loss: 2.5575 - val_acc: 0.1895
Epoch 2/10
15998/15998 [==============================] - 24s - loss: 2.3871 - acc: 0.1875 - val_loss: 2.0241 - val_acc: 0.3221
Epoch 3/10
15998/15998 [==============================] - 24s - loss: 2.0195 - acc: 0.2805 - val_loss: 1.7412 - val_acc: 0.4204
Epoch 4/10
15998/15998 [==============================] - 25s - loss: 1.7680 - acc: 0.3511 - val_loss: 1.4873 - val_acc: 0.4951
Epoch 5/10
15998/15998 [==============================] - 25s - loss: 1.5797 - acc: 0.4110 - val_loss: 1.3147 - val_acc: 0.5819
Epoch 6/10
15998/15998 [==============================] - 25s - loss: 1.3931 - acc: 0.4831 - val_loss: 1.1932 - val_acc: 0.6047
Epoch 7/10
15998/15998 [==============================] - 24s - loss: 1.2762 - acc: 0.5314 - val_loss: 1.0887 - val_acc: 0.6374
Epoch 8/10
15998/15998 [==============================]

In [15]:
get_pooling_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[1].output])

In [16]:
new_X_train = []
for raw_doc_features in X_train:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1),0])
    new_X_train.append(layer_output[0].ravel())
new_X_train = np.vstack(new_X_train)

new_X_train.shape

(15998, 100)

In [17]:
new_X_val = []
for raw_doc_features in X_val:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1),0])
    new_X_val.append(layer_output[0].ravel())
new_X_val = np.vstack([new_X_val])

new_X_val.shape

(3999, 100)

In [18]:
# LINEAR SVM

tol_range = np.logspace(-10,-4,7)

clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.LinearSVC()),
]),n_jobs=-1)
parameters = [
    { 
          "estimator__clf__penalty": ["l1","l2"],
          "estimator__clf__dual":[False],
          "estimator__clf__multi_class":["ovr"],
          "estimator__clf__tol": tol_range,

    }
]

In [19]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_X_train)    
    Y_pred_val = clf.predict(new_X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score

        best_grid = g

CPU times: user 668 ms, sys: 252 ms, total: 920 ms
Wall time: 35.7 s
train accuracy: 0.43417927240905113
val accuracy: 0.3938484621155289
grid: {'estimator__clf__multi_class': 'ovr', 'estimator__clf__dual': False, 'estimator__clf__tol': 1e-10, 'estimator__clf__penalty': 'l1'}

CPU times: user 724 ms, sys: 668 ms, total: 1.39 s
Wall time: 35.3 s
train accuracy: 0.43417927240905113
val accuracy: 0.3938484621155289
grid: {'estimator__clf__multi_class': 'ovr', 'estimator__clf__dual': False, 'estimator__clf__tol': 1.0000000000000001e-09, 'estimator__clf__penalty': 'l1'}

CPU times: user 696 ms, sys: 708 ms, total: 1.4 s
Wall time: 34.7 s
train accuracy: 0.43417927240905113
val accuracy: 0.3938484621155289
grid: {'estimator__clf__multi_class': 'ovr', 'estimator__clf__dual': False, 'estimator__clf__tol': 1e-08, 'estimator__clf__penalty': 'l1'}

CPU times: user 684 ms, sys: 716 ms, total: 1.4 s
Wall time: 35 s
train accuracy: 0.43417927240905113
val accuracy: 0.3938484621155289
grid: {'estimat

In [20]:
# RBF SVM
# http://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use

sc= preprocessing.StandardScaler()
new_new_X_train = sc.fit_transform(new_X_train)
new_new_X_val = sc.transform(new_X_val)

clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.SVC()),
]),n_jobs=-1)

C_range = np.logspace(0, 7, 8)
gamma_range = np.logspace(-6, 2, 9)
parameters = [{
    'estimator__clf__gamma':gamma_range,
    'estimator__clf__C':C_range
}]

best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(new_new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_new_X_train)    
    Y_pred_val = clf.predict(new_new_X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

CPU times: user 548 ms, sys: 368 ms, total: 916 ms
Wall time: 24 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__C': 1.0, 'estimator__clf__gamma': 9.9999999999999995e-07}

CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s
Wall time: 26.5 s
train accuracy: 0.003937992249031129
val accuracy: 0.003500875218804701
grid: {'estimator__clf__C': 1.0, 'estimator__clf__gamma': 1.0000000000000001e-05}

CPU times: user 992 ms, sys: 0 ns, total: 992 ms
Wall time: 26.5 s
train accuracy: 0.049381172646580825
val accuracy: 0.04551137784446112
grid: {'estimator__clf__C': 1.0, 'estimator__clf__gamma': 0.0001}

CPU times: user 912 ms, sys: 0 ns, total: 912 ms
Wall time: 24.3 s
train accuracy: 0.24271783972996624
val accuracy: 0.2145536384096024
grid: {'estimator__clf__C': 1.0, 'estimator__clf__gamma': 0.001}

CPU times: user 984 ms, sys: 0 ns, total: 984 ms
Wall time: 24.8 s
train accuracy: 0.5166895861982748
val accuracy: 0.45186296574143536
grid: {'estimator__clf__C': 1.0, 'estimator__cl

KeyboardInterrupt: 